In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

MODEL = "llama3.2"

# Optional headers to avoid request blocks
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}


class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""


system_prompt = """You are an assistant that analyzes the contents of a website 
and provides a short summary, ignoring navigation text. Respond in markdown."""


def user_prompt_for(website):
    return f"""You are looking at a website titled {website.title}.
The contents of this website are as follows. Please provide a short summary in markdown. 
If it includes news or announcements, summarize these too.

{website.text}
"""


def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(website)}
        ]
    )
    return response['message']['content']


def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))


# Example usage
display_summary("https://edwarddonner.com")
